In [1]:
#!pip install -U selenium
#!pip install maya

In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import pickle
import time
from datetime import datetime
import maya
import re
import os
import json
import pickle
from glob import glob

In [57]:
# get a reference to the download menu. This will run before the page has 
# finished loading, so we stick it in a while loop and just keep looping
# until we're successful.
def find_by_id(element_id):
    while True:
        try:
            target = driver.find_element_by_id(element_id)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target

def find_by_class(element_class):
    counter_broke = 0
    while True:
        try:
            target = driver.find_element_by_class_name(element_class)
        except NoSuchElementException:
            time.sleep(0.2)
            if (counter_broke > 1):
                break
            counter_broke += 1
            continue
        else:
            break
    return target

def get_link(link):
    driver.implicitly_wait(2.5)
    try:
        driver.get(link)
    except TimeoutException:
        print("problem loading: " + link)
        driver.refresh()
            

In [58]:
with open("dep_list.txt", "rb") as fp:   # Unpickling
    dep_list = pickle.load(fp)
print(dep_list[5000:5100])

['eo-learn-features', 'tf-conversions', 'nmslib', 'pelican-render-math', 'pymarkovchain', 'asana', 'beholder', 'percy', 'ghp-import2', 'pyzbar', 'elfi', 'azureml-model-management-sdk', 'jep', 'mimeparse', 'rqt-dep', 'gprof2dot', 'bmcs', 'web3', 'nbpdfexport', 'mirapy', 'cleo', 'cltk', 'wordninja', 'cassandra-driver', 'planar', 'baobao', 'pykneer', 'mkl_fft', 'fog', 'pyformulas', 'crdp', 'nptdms', 'pysf', 'rx', 'matplotlib-base', 'multiprocessing-on-dill', 'featuretools_tsfresh_primitives', 'shutilwhich', 'chart_studio.plotly', 'postcodes-io-api', 'roscreate', 'audio.coders', 'static3', 'pygmsh', 'platypus-opt', 'tvb-library', 'azure-mgmt-security', 'cn2an', 'docrepr', 'simple-salesforce', 'fusepy', 'orange3-associate', 'pyats.examples', 'rqt-graph', 'annotationframeworkclient', 'python-pam', 'pymssql', 'ml-insights', 'fast_pagerank', 'efficientnet-pytorch', 'apng', 'optimparallel', 'rasa-sdk', 'tf-nightly-gpu-2.0-preview', 'roslib', 'azure-mgmt-netapp', 'django-class', 'python-redis-ca

In [59]:
len(dep_list)

5190

In [60]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') -- using headless mode currently leads to NoSuchElementException 
driver = webdriver.Chrome('/Users/kevingao/Documents/beaut/chromedriver', options=options)

In [61]:
def split_repo_stat(s):
    return int(s.split(" ")[-1].replace(",", ""))

def page_scrape(pack_df, name):
    
    
    pack_row = {}
   
    pack_row["library"] = name

    
    # LIBRARIES.IO
    
    # Stats
    link = "https://libraries.io/pypi/" + name
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)  

        page_region = find_by_class("container")
        content_region = page_region.find_elements_by_xpath("//*[@class='col-md-4 sidebar']")

        stats_region = content_region[0]
        sourcerank = stats_region.find_element_by_tag_name("h2")
        pack_row["SourceRank"] = sourcerank.text.split('\n')[1]

        stats = stats_region.find_elements_by_tag_name("dl")[1]
        split_stats = stats.text.split('\n')
        for i in np.arange(len(split_stats) // 2):
            pack_row[stats.text.split('\n')[i * 2]] = stats.text.split('\n')[i * 2 + 1]

        # Versions
        orig_v_link = link + "/versions"
        v_link = orig_v_link
        v_index = 1
        ver_num, ver_date = [], []
        #driver.get(link + "/versions")
        
        while True:
            get_link(v_link)
            time.sleep(np.random.rand() * 3 + 2)
            page_region = find_by_class("col-sm-8")
            try:
                tab_ver = page_region.find_element_by_tag_name("tbody")
            except:
                break
            ind_ver = tab_ver.find_elements_by_tag_name("tr")

            for i in ind_ver:
                front = i.text.split(" Br")[0]
                split_v_d = front.split(" ", 1)
                ver_num.append(split_v_d[0])
                ver_date.append(maya.parse(split_v_d[1]).datetime())
            
            v_index += 1
            v_link = orig_v_link + "?page=" + str(v_index)

        pack_row["Version Number"] = ver_num
        pack_row["Version Time"] = ver_date
    except:
        pass
    
    # PYPI
    link = "https://pypi.org/project/" + name  
    
    try:
        get_link(link)
        time.sleep(np.random.rand() * 3 + 2)
        #driver.get(link)
        page_region = find_by_class("github-repo-info")
        boxes = page_region.find_elements_by_tag_name("li")

        pack_row["stars"] = split_repo_stat(boxes[0].text)
        pack_row["forks"] = split_repo_stat(boxes[1].text)
        pack_row["Open Issues/PRs"] = split_repo_stat(boxes[2].text)
    except:
        pass

    
    # Appending
    pack_df = pack_df.append(pack_row, ignore_index=True)
    
    print(name)
    return pack_df
    
    
'''
package_df = pd.DataFrame()

name = "scipy"
page_scrape(package_df, name)
'''

'\npackage_df = pd.DataFrame()\n\nname = "scipy"\npage_scrape(package_df, name)\n'

In [62]:
start = 4000
end = 5190


package_df = pd.DataFrame(columns=["library"])

for i in np.arange(start, end):
    package_df = page_scrape(package_df, dep_list[i])

python-debian
keyrings-alt
menpofit
dash_daq
pyats.templates
click_log
neotime
vegasflow
miepython
azure-mgmt-cdn
datreant.core
stemming
sacrebleu
event-model
nbforms
django-compressor
pygeodesy
paste
pygenometracks
azure-mgmt-media
kytea
python-libtorrent
pyats.datastructures
pycifrw
pyats.robot
webweb
fcsparser
aiohttp-cors
locustio
numpy.matlib
pythonturtle
flavio
protobuf3-to-dict
osm-conflate
qt-gui
flowws
python-delairstack
scikit-vectors
torch-summary
klampt
locuspeerexplorer
tf2-py
gender-guesser
montage_wrapper
boxx
bigml
ipystata
elm_kernel
vaex-hdf5
pylcs
aicsfeature
dash-table-experiments
pymarkdown
autopy
lsanomaly
uber-rides
torchani
sphinxjp.themes.solarized
azure-cli-telemetry
snowflake-connector-python
qsymm
labml_helpers
ufal.morphodita
webp
wave
pyteaser
gtar
scp
html2json
multimethodic
catkin
portfoliotools
pgeocode
dlisio
fn
flask-talisman
makey
onnx-tf
pandarallel
rosmsg
nb2pdf
seabird
suitcase-tiff
mdtraj
regional
pyls
purplex
dubins
scipy.io
bondpy
dendropy
genl

nn_builder
repo2data
pdbpp
mathparse
azure-mgmt-msi
pymsk
colorednoise
mlpack
scipy.fftpack
azureml-contrib-fairness
dataverk
azure-mgmt-monitor
azure-cognitiveservices-speech
azure-mgmt-recoveryservices
gpmap
urlparse2
pygubu
swagger-spec-validator
netwulf
vulture
brythonmagic
itk-spcn
ratelimit
pinyin
nbparameterise
codacy-coverage
llist
stylo
audio.psychoacoustics
pyats.connections
azure-mgmt-core
brainiak
pypubsub
mousai
pydna
pymoose
bezier
pylint-flask
xkcdpass
truth-table-generator
jinja2-highlight
geomagindices
gradememaybe
munkres
datastep
nolearn
azure-mgmt-billing
pylint-django
rantanplan
ivolat3
genie.libs.telemetry
rq
nbdiff
steppy
coinmetrics
rasterfairy
prettyprint
azure-mgmt-redhatopenshift
spark-sklearn
instapy-chromedriver
eland
bumps
diffpy-srfit
xmljson
azure-mgmt-relay
virtualfish
express
jupyterlab-code-formatter
kmeans-smote
sat-search
bravado
pyschedule
pyowm
russian-tagsets
simpleeval
mail-parser
azure-ai-textanalytics
macholib
yamlordereddictloader
fanalysis
c

In [63]:
df = package_df.copy(deep=True)

In [64]:
df

,library,Dependencies,Dependent packages,Dependent repositories,First release,Latest release,SourceRank,Total releases,Version Number,Version Time,Open Issues/PRs,forks,stars,Contributors,Forks,Repository size,Stars,Watchers
0,python-debian,2,14,1.28K,"Apr 26, 2009","Dec 19, 2020",11,16,"[0.1.39, 0.1.38, 0.1.37, 0.1.36, 0.1.35, 0.1.3...","[2020-12-19 01:17:00+00:00, 2020-10-04 04:45:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,keyrings-alt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,22.0,16.0,NaN,NaN,NaN,NaN,NaN
2,menpofit,0,0,6,"Nov 8, 2014","Jan 14, 2021",6,6,"[0.7.0, 0.6.1, 0.6.0, 0.5.0, 0.3.0, 0.1.0a1]","[2021-01-14 21:00:00+00:00, 2019-07-27 19:49:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dash_daq,0,23,84,"Jan 7, 2019","Apr 27, 2020",13,23,"[0.5.0, 0.4.0, 0.3.3, 0.3.2, 0.3.1, 0.3.0, 0.2...","[2020-04-27 14:23:00+00:00, 2020-03-04 22:13:0...",63.0,30.0,118.0,15,25,6.29 MB,111,26
4,pyats.templates,3,1,2,"Nov 17, 2017","Jun 25, 2019",5,8,"[19.6, 19.5.0, 19.4.0, 19.0, 5.1.0, 5.0.0, 4.1...","[2019-06-25 16:51:00+00:00, 2019-05-28 16:05:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,pyats.aetest,9,3,2,"Nov 27, 2017",about a month ago,5,40,"[21.5.1, 21.5, 21.4, 21.3.1, 21.3, 21.2, 21.1,...","[2021-05-26 12:44:00+00:00, 2021-05-25 23:58:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1186,dicrivers,0,0,0,"Jun 6, 2019","Jun 6, 2019",5,1,[0.0],[2019-06-06 16:37:00+00:00],NaN,NaN,NaN,1,0,6.1 MB,1,1
1187,airr,0,4,1,"May 3, 2018","Oct 13, 2020",9,5,"[1.3.1, 1.3.0, 1.2.1, 1.2.0, 1.1.0]","[2020-10-13 21:51:00+00:00, 2020-06-01 20:54:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1188,python-afl,0,1,8,"Apr 16, 2015","Oct 6, 2020",6,17,"[0.7.3, 0.7.2, 0.7.1, 0.7, 0.6.1, 0.6, 0.5.5, ...","[2020-10-06 18:22:00+00:00, 2019-02-15 14:38:0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
package_df[package_df["Repository size"].fillna("0").str.contains("yte")]

,library,Dependencies,Dependent packages,Dependent repositories,First release,Latest release,SourceRank,Total releases,Version Number,Version Time,Open Issues/PRs,forks,stars,Contributors,Forks,Repository size,Stars,Watchers
415,myturtle,0,0,0,"Apr 30, 2020","Apr 30, 2020",5,1,[0.1],[2020-04-30 14:49:00+00:00],NaN,NaN,NaN,2,0,1000 Bytes,0,1
780,muffle,0,1,1,"Nov 7, 2015","Nov 7, 2015",7,1,[1.0.0],[2015-11-07 19:22:00+00:00],0.0,0.0,1.0,1,0,0 Bytes,0,1


In [66]:
def col2int(s):
    multiplier = 1
    if (type(s) != str):
        return
    s = s.replace(",", "")
    if (s[-1] == "K"):
        multiplier = 1000
        s = s[:-1]
    if (s[-1] == "B" or s[-1] == "s"):
        if (s[-2] == "MB"):
            multiplier = 1
        if (s[-2] == "KB"):
            multiplier = (10 ** -3)
        if (s[-2] == "es"):
            multiplier = (10 ** -6)
        if (s[-2] == "GB"):
            multiplier = (10 ** 3)
        s = s.split(" ")[0]
        
    return float(s) * multiplier

In [67]:
package_col_need = ['Contributors', 'Dependencies', 'Dependent packages',
       'Dependent repositories', 'Forks', 'Repository size', 'SourceRank', 'Stars',
       'Total releases', 'Watchers']

for col in package_col_need:
    package_df[col] = package_df[col].apply(lambda x: col2int(x))

In [68]:
def two_col_combine_not_null(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]

In [69]:
package_df["Stars"] = package_df[["stars", "Stars"]].apply(two_col_combine_not_null, axis=1)
package_df["Forks"] = package_df[["forks", "Forks"]].apply(two_col_combine_not_null, axis=1)

In [70]:
package_df = package_df.drop(["stars", "forks"], axis=1)

In [71]:
package_df

,library,Dependencies,Dependent packages,Dependent repositories,First release,Latest release,SourceRank,Total releases,Version Number,Version Time,Open Issues/PRs,Contributors,Forks,Repository size,Stars,Watchers
0,python-debian,2.0,14.0,1280.0,"Apr 26, 2009","Dec 19, 2020",11.0,16.0,"[0.1.39, 0.1.38, 0.1.37, 0.1.36, 0.1.35, 0.1.3...","[2020-12-19 01:17:00+00:00, 2020-10-04 04:45:0...",NaN,NaN,NaN,NaN,NaN,NaN
1,keyrings-alt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,22.0,NaN,16.0,NaN
2,menpofit,0.0,0.0,6.0,"Nov 8, 2014","Jan 14, 2021",6.0,6.0,"[0.7.0, 0.6.1, 0.6.0, 0.5.0, 0.3.0, 0.1.0a1]","[2021-01-14 21:00:00+00:00, 2019-07-27 19:49:0...",NaN,NaN,NaN,NaN,NaN,NaN
3,dash_daq,0.0,23.0,84.0,"Jan 7, 2019","Apr 27, 2020",13.0,23.0,"[0.5.0, 0.4.0, 0.3.3, 0.3.2, 0.3.1, 0.3.0, 0.2...","[2020-04-27 14:23:00+00:00, 2020-03-04 22:13:0...",63.0,15.0,30.0,6.29,118.0,26.0
4,pyats.templates,3.0,1.0,2.0,"Nov 17, 2017","Jun 25, 2019",5.0,8.0,"[19.6, 19.5.0, 19.4.0, 19.0, 5.1.0, 5.0.0, 4.1...","[2019-06-25 16:51:00+00:00, 2019-05-28 16:05:0...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,pyats.aetest,9.0,3.0,2.0,"Nov 27, 2017",about a month ago,5.0,40.0,"[21.5.1, 21.5, 21.4, 21.3.1, 21.3, 21.2, 21.1,...","[2021-05-26 12:44:00+00:00, 2021-05-25 23:58:0...",NaN,NaN,NaN,NaN,NaN,NaN
1186,dicrivers,0.0,0.0,0.0,"Jun 6, 2019","Jun 6, 2019",5.0,1.0,[0.0],[2019-06-06 16:37:00+00:00],NaN,1.0,0.0,6.10,1.0,1.0
1187,airr,0.0,4.0,1.0,"May 3, 2018","Oct 13, 2020",9.0,5.0,"[1.3.1, 1.3.0, 1.2.1, 1.2.0, 1.1.0]","[2020-10-13 21:51:00+00:00, 2020-06-01 20:54:0...",NaN,NaN,NaN,NaN,NaN,NaN
1188,python-afl,0.0,1.0,8.0,"Apr 16, 2015","Oct 6, 2020",6.0,17.0,"[0.7.3, 0.7.2, 0.7.1, 0.7, 0.6.1, 0.6, 0.5.5, ...","[2020-10-06 18:22:00+00:00, 2019-02-15 14:38:0...",NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
pack_json = package_df.to_json()
with open("scraper_" + str(start) + "_" + str(end - 1) + ".txt",'w') as f:
    json.dump(pack_json, f)

---------------------

In [262]:
stopper

Combining csvs from above.

In [9]:
files = glob("scraper_*")

In [10]:
print(files)

['scraper_3000_3999.txt', 'scraper_2000_2999.txt', 'scraper_4000_5189.txt', 'scraper_0_999.txt', 'scraper_1000_1999.txt']


In [11]:
data = pd.DataFrame()

for file in files:
    with open(file) as f:
        reloaded_example = json.load(f)
    data = data.append(pd.read_json(reloaded_example))

In [18]:
data = data.reset_index().drop("index", axis=1)

In [20]:
data

,library,Contributors,Dependencies,Dependent packages,Dependent repositories,First release,Forks,Latest release,Open Issues/PRs,Repository size,SourceRank,Stars,Total releases,Version Number,Version Time,Watchers
0,cutecharts,2.0,1.0,0.0,0.0,"Oct 12, 2019",56.0,"Oct 14, 2019",2.0,81.10,11.0,503.0,3.0,"[1.2.0, 1.1.0, 1.0.0]","[1571038080000, 1571034300000, 1570870740000]",18.0
1,postgres,6.0,2.0,2.0,192.0,"Aug 8, 2013",17.0,"Oct 19, 2019",16.0,239.00,12.0,55.0,12.0,"[3.0.0, 2.2.2, 2.2.1, 2.2.0, 2.1.2, 2.1.1, 2.1...","[1571513400000, 1536735600000, 1444434720000, ...",8.0
2,gpustat,10.0,0.0,68.0,29.0,"Aug 19, 2016",202.0,"Jan 2, 2021",23.0,271.00,16.0,2479.0,11.0,"[1.0.0b1, 0.6.0, 0.5.0, 0.4.1, 0.4.0, 0.3.2, 0...","[1609567140000, 1563776220000, 1536551400000, ...",39.0
3,df2gspread,6.0,0.0,2.0,3.0,"Jan 19, 2016",26.0,"Jan 3, 2019",12.0,116.00,10.0,117.0,21.0,"[1.0.4, 1.0.3, 1.0.2, 1.0.1, 1.0.0, 0.2.5, 0.0...","[1546516500000, 1525855980000, 1525337280000, ...",8.0
4,pyprism,2.0,0.0,0.0,1.0,"Apr 4, 2018",14.0,"Jul 1, 2020",11.0,25.20,9.0,23.0,5.0,"[1.0.4, 1.0.3, 1.0.2, 1.0.1, 1.0.0]","[1593632700000, 1522935600000, 1522892040000, ...",6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5185,dwave-ocean-sdk,13.0,18.0,3.0,20.0,"May 11, 2018",101.0,"Jan 25, 2021",14.0,8.62,12.0,268.0,29.0,"[3.3.0, 3.2.0, 3.1.1, 3.1.0, 3.0.1, 3.0.0, 3.0...","[1611534180000, 1608150060000, 1602127440000, ...",32.0
5186,mpi4py,0.0,0.0,120.0,376.0,"Oct 15, 2006",20.0,"Nov 4, 2019",NaN,10.80,15.0,NaN,13.0,"[3.0.3, 3.0.2, 3.0.1, 3.0.0, 2.0.0, 1.3.1, 0.6...","[1572872700000, 1560254940000, 1550239440000, ...",48.0
5187,simple_colors,1.0,0.0,1.0,1.0,"Nov 23, 2018",0.0,"Jan 14, 2019",1.0,4.88,6.0,2.0,6.0,"[0.1.5, 0.1.4, 0.1.3, 0.1.2, 0.1.1, 0.1.0]","[1547456940000, 1547358360000, 1547357880000, ...",1.0
5188,altair-viewer,2.0,2.0,4.0,0.0,"Dec 29, 2019",4.0,"Mar 31, 2020",1.0,2.26,11.0,46.0,6.0,"[0.3.0, 0.2.1, 0.2.0, 0.1.2, 0.1.1, 0.1.0]","[1585658640000, 1583295120000, 1579097280000, ...",3.0


In [19]:
pack_final = data.to_json()
with open("scraper_final.txt",'w') as f:
    json.dump(pack_final, f)